In [ ]:
!pip install transformers

In [ ]:
import torch
import transformers as ppb
import pandas as pd
import numpy as np

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cse6242project-417410'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [ ]:
bucket_name = 'team054'
# Download the file from the given Google Cloud Storage bucket.
!gsutil cp gs://team054/books_enriched.csv books_enriched.csv

Copying gs://team054/books_enriched.csv...
-
Operation completed over 1 objects/13.0 MiB.                                     


In [ ]:
import pandas as pd
from ast import literal_eval
#books_df = pd.read_csv('/content/books_enriched.csv')#---if uploading through the bucket way,then uncomment this

books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/books_enriched.csv')

#books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/combined_processed_files.txt')  --- Terry's file

books_df = books_df[:1000]

In [ ]:
books_df['short_description'] = books_df.description.str.slice(0,512)

In [ ]:


df = books_df.short_description

In [ ]:
books_df2 = pd.read_table('/content/drive/MyDrive/CSE6242_project/gutenberg_books.txt', index_col=[0])

In [ ]:
books_df2['short_description'] = books_df.description

In [ ]:
# add all rows of books_df2['short_description'] to df

df = pd.concat([df, books_df2['short_description']])

In [ ]:
tokenized = df.apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

# Embed Text

In [ ]:
#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#PYTORCH_NO_CUDA_MEMORY_CACHING=1
##set CUDA_VISIBLE_DEVICES=""

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [23]:
embeddings = last_hidden_states[0].cpu().data[:,0,:].numpy()

In [ ]:
df.shape, embeddings.shape

((42723,), (42723, 768))

# Save Embeddings

In [24]:
embeddings_pd_df = pd.DataFrame({'book_id': range(df.shape[0])})
embeddings_pd_df['title'] = books_df['title']
embeddings_pd_df['embedding'] = embeddings.tolist()

In [25]:

embeddings_pd_df['embedding'] = embeddings_pd_df['embedding'].apply(lambda x: np.array(x))


In [26]:
embeddings_pd_json= embeddings_pd_df.to_json()

In [27]:
#from google.colab import drive
#drive.mount('/content/drive')
df.to_json('/content/drive/MyDrive/CSE6242_project/embeddings_pd_json.json')

In [ ]:
#!gsutil cp embeddings_pd_txt gs://team054

In [ ]:
#import datalab.storage as gcs
#gcs.Bucket('team054').item('to/embeddings_pd_json.json').write_to(simple_dataframe.to_json(),'text/json')

In [ ]:
#from google.cloud import storage

#def write_to_cloud(buffer):
 #   client = storage.Client()
  #  bucket = client.get_bucket('team054')
  #  blob = bucket.blob('gs://team054/embeddings_pd_json.json')
 #   blob.upload_from_file(buffer)

In [ ]:
#from google.cloud import storage

#def create_newfolder(bucket_name, destination_folder_name):
 #   storage_client = storage.Client()
 #   bucket = storage_client.get_bucket('team054')
  #  blob = bucket.blob('output')

  #  blob.upload_from_file('')

   # print('Created {} .'.format(destination_folder_name))

In [28]:
embeddings_pd_df.head()

,book_id,title,embedding
0,0,"The Hunger Games (The Hunger Games, #1)","[0.1194697842001915, 0.057542070746421814, 0.3..."
1,1,Harry Potter and the Sorcerer's Stone (Harry P...,"[-0.374781996011734, -0.272121787071228, 0.131..."
2,2,"Twilight (Twilight, #1)","[0.12502777576446533, -0.7245572805404663, -0...."
3,3,To Kill a Mockingbird,"[-0.43787628412246704, -0.29153966903686523, -..."
4,4,The Great Gatsby,"[-0.40598979592323303, -0.47040653228759766, -..."


# Train KNN

In [29]:
# cell edited by Tyler (3/24/2024)

from sklearn.neighbors import NearestNeighbors

# fit knn model
embedding_vals = np.array(embeddings_pd_df['embedding'].values.tolist())
k = 5
nn_model = NearestNeighbors(n_neighbors=k, algorithm="ball_tree").fit(embedding_vals)

In [30]:
# save the trained model to mydrive as pickle file
import cloudpickle

with open('/content/drive/MyDrive/CSE6242_project/knn_model.pkl', 'wb') as f:
  cloudpickle.dump(nn_model, f)

In [31]:
#import blosc
!pip install blosc

In [41]:
#import pickle as pkl

#import blosc

#def pickle(nn_model, filepath):
    """
    Pickle, compress, and save it to a file.

    Parameters
    ----------
    obj : object
        Any python object to be pickled.
    filepath : str
        File path destination.
    """
   # arr = pkl.dumps(nn_model, -1)
   # with open('/content/drive/MyDrive/CSE6242_project/knn_model_compress.pkl', "wb") as f:
   #     s = 0
    #    while s < len(arr):
    #        e = min(s + blosc.MAX_BUFFERSIZE, len(arr))
    #        carr = blosc.compress(arr[s:e], typesize=8)
    #        f.write(carr)
    #        s = e
    #    pickle_blosc.dump(knn_model_compress, f)

In [42]:
!pip install bz2file

  Preparing metadata (setup.py) ... done
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6868 sha256=f47922686b89b0b979ef316b39223ee1d44bbe614a1301cca6531f2bae15459b
  Stored in directory: /root/.cache/pip/wheels/69/ee/f7/6fccd10cb65421ba2da64fa6caf8ee7fbae0059884af8c8587
Successfully built bz2file


In [43]:
import bz2file as bz2
import pickle

In [50]:
def compressed_pickle(title, data):

  with bz2.BZ2File('/content/drive/MyDrive/CSE6242_project/knn_model_compress.pkl' + '.pbz2', 'w') as f:
    pickle.dump(nn_model, f)

In [52]:
compressed_pickle('filename', nn_model)